In [1]:
from qtuum.api_wrappers import QuantinuumAPI as QAPI
import time

machine = 'H1-2E'

qapi = QAPI(machine=machine)
print('Machine status:', qapi.machine, 'is', qapi.status())


Machine status: H1-2E is reserved


In [2]:
from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.circuit.display import render_circuit_jupyter

machine = 'H1-2E'
backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email:  jha3@llnl.gov
Enter your Quantinuum password:  ········


In [4]:
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox
import numpy as np

u1 = np.asarray([[2/3, (-2+1j)/3],
                 [(2+1j)/3, 2/3]])
u1box = Unitary1qBox(u1)

u2 = np.asarray([[0, 1, 0, 0],
                 [0, 0, 0, -1],
                 [1, 0, 0, 0],
                 [0, 0, -1j, 0]])
u2box = Unitary2qBox(u2)

circ = Circuit(3)
circ.add_unitary1qbox(u1box, 0)
circ.add_unitary2qbox(u2box, 1, 2)
circ.add_unitary1qbox(u1box, 2)
circ.add_unitary2qbox(u2box, 1, 0)

[Unitary1qBox q[0]; Unitary2qBox q[1], q[2]; Unitary2qBox q[1], q[0]; Unitary1qBox q[2]; ]

In [5]:
de = 2*np.pi/3.0
F3 = 1.0/np.sqrt(3.0) * np.asarray([
    [1,              1,              1,          0],
    [1,  np.exp(1j*de), np.exp(-1j*de),          0],
    [1, np.exp(-1j*de),  np.exp(1j*de),          0],
    [0,              0,              0, np.sqrt(3)]
])
F3_gate = Unitary2qBox(F3)

In [6]:
# Grover Diffusion: 8 gates, 6 U + 2 CX
Us = 1/3.0 * np.asarray([
    [-1, 2, 2, 0],
    [2, -1, 2, 0],
    [2, 2, -1, 0],
    [0, 0, 0,  3]
])
Us_gate = Unitary2qBox(Us)

In [7]:
# Oracle:
omega = 1
Uw_array = np.asarray([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
Uw_array[omega,omega] = -1
Uw_gate = Unitary2qBox(Uw_array)

In [8]:
circ = Circuit(2)
circ.add_unitary2qbox(F3_gate, 0, 1)
circ.add_unitary2qbox(Uw_gate, 0, 1)
circ.add_unitary2qbox(Us_gate, 0, 1)
circ.measure_all()

[Unitary2qBox q[0], q[1]; Unitary2qBox q[0], q[1]; Unitary2qBox q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [9]:
#from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(circ)

In [10]:
compiled_circuit = backend.get_compiled_circuit(circ, optimisation_level=2)

In [11]:
n_shots = 128
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

6.4848

In [12]:
render_circuit_jupyter(compiled_circuit)

In [13]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "job", "submit-date": "2023-07-20T20:51:07.163258", "result-date": null, "queue-position": null, "cost": null, "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [14]:
result = backend.get_result(handle)

result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [ 64]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [ 64]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [ 64]
 [128]
 [128]
 [128]
 [128]
 [ 64]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [  0]
 [128]
 [128]
 [ 64]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]
 [128]],state=None,unitary=None,density_mat

In [15]:
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.03125, (0, 1): 0.9296875, (1, 0): 0.0390625}
